# Storing scraped articles

In [1]:
import numpy as np
import pandas as pd
import requests
import re
import sqlite3
import time
import json
import openai
import redis

In [2]:
r = redis.Redis("localhost", 5211, decode_responses=True)

In [3]:
db = sqlite3.connect("scraping.db")

In [4]:
cur = db.execute("select count(*) from article where content is null")
db.commit()
for row in cur:
    print(row)

(84380,)


In [7]:
n = 14716
while True:
    try:
        if (data := r.rpop("to_store_tertam")) is None:
            print(f"{n} Nothing to store, idling...", end="\r")
            time.sleep(1)
        else:
            data = json.loads(data)
            
            if len(data["id"]) == 0:
                print("ERROR, NO ID", end="\r")
                continue
            if len(data["content"]) == 0:
                print("ERROR, NO CONTENT", end="\r")
                continue

            n += 1
            print(str(n) + "      ", end="\r")

            db.execute(f'update article set content = ? where id = ?', (data["content"], data["id"]))
            db.commit()
    except KeyboardInterrupt as ki:
        db.commit()

In [6]:
db.commit()

In [ ]:
r.llen("to_store_tertam")

0